# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [42]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
# set the parameters to run the script
directories = ['/media/elea/ssd/NPP/550x600']
calib_directory = '/media/elea/ssd/NPP/calib'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = False

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'both'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550, 600, 650]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz

# NB: processing time without PDDN
# 'no_viz': 0.71s
# 'default', save_pdf_figs False: 2.25s
# 'default', save_pdf_figs True: 3.60s
# 'full', save_pdf_figs False: 3.95s
# 'full', save_pdf_figs True: 8.07s
processing_mode = 'default'
save_pdf_figs = True

# define if the wavelenghts should be aligned before processing - and used for the computation
align_wls = True

parameters = batch_processing.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, processing_mode = processing_mode, run_all = run_all, 
                                save_pdf_figs = save_pdf_figs, align_wls = align_wls)

In [46]:
%%time
times = batch_processing.batch_process_master(parameters)
times

processing without PDDN...
Aligning wavelengths...
Aligning wavelenght: 600nm


100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 13294.15it/s]


Aligning wavelenght: 650nm


100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 17924.38it/s]


Aligning wavelengths done.



...

100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 8168.07it/s]


Aligning wavelengths done.



0it [00:00, ?it/s]

processing with PDDN done.
CPU times: user 5.37 s, sys: 446 ms, total: 5.82 s
Wall time: 3.67 s


{'total': 0}

In [25]:
%%time

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
times = batch_processing.batch_visualization(parameters)
times

CPU times: user 14.5 s, sys: 28.9 ms, total: 14.5 s
Wall time: 14.5 s


14.527570009231567

In [15]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
directories = ['/media/elea/ssd/NPP/550x600']

PDDN_mode = 'both'
run_all = False
align_wavelengths.align_wavelenghts(directories, PDDN_mode, run_all)

Aligning wavelenght: 600nm


  0%|                                                     | 0/1 [00:03<?, ?it/s]

 
 >> Exported X3D .cod file as: /media/elea/ssd/NPP/550x600/2022-10-11_T_HORAO-55-BF_FR_S1_1/raw_data/600nm/600_Intensite_aligned.cod


FileNotFoundError: [Errno 2] No such file or directory: '/media/elea/ssd/NPP/550x550/2022-10-11_T_HORAO-55-BF_FR_S1_1/raw_data/550nm/550_Intensite.cod'

In [39]:
# add the calibration matrices (A.cod and W.cod) to the folder for processing
# needed by Christopher
from processingmm.addons import add_calibration
add_calibration.add_calibration(directories)

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 15.59it/s]

No calibration files in 600nm
